In [1]:
train_size = 47500

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import xgboost as xgb
import time
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [3]:
np.set_printoptions(precision=9)
np.set_printoptions(linewidth=np.inf)

In [4]:
X = np.load('../../X_train.npz')['arr_0']
Y = np.load('../../Y_train.npz')['arr_0']
TX = np.load('../../X_test.npz')['arr_0']

In [5]:
XTX = np.concatenate((X, TX), axis=0)

In [ ]:
X = None
TX = None

In [ ]:
XTX_square = XTX ** 2
XTX_cumsum = np.concatenate((np.cumsum(XTX[:, :5000], axis=1), np.cumsum(XTX[:, 5000:], axis=1)), axis=1)
XTX_all = np.concatenate((XTX, XTX_square, XTX_cumsum), axis=1)

In [ ]:
XTX_square = None
XTX_cumsum = None
XTX = None

In [ ]:
XTX_all.shape

In [ ]:
X, X_test = XTX_all[:train_size], XTX_all[train_size:]
print(X.shape, X_test.shape)

In [ ]:
# calculate type 1 error
def err1(y_pred, y_true):
    return 't1e', np.sum(1.0 * np.abs(y_pred - y_true.get_label())) / len(y_pred)

# calculate type 2 errr
def err2(y_pred, y_true):
    return 't2e', np.sum(np.abs(y_pred - y_true.get_label()) / y_true.get_label()) / len(y_pred)
scorer = {'t1':err1, 't2':err2}

In [ ]:
params = {'booster':'gbtree', 'learning_rate':0.05, 'max_depth':5, 'n_estimators':500, 'tree_method':'hist', 'n_jobs':-1, 'subsample':}

In [ ]:
ty = []
vy = []
feature_imp = []

for y_id in range(3):
    kf = KFold(n_splits=5, shuffle=False)
    tty = []
    tvy = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx, y_id], Y[val_idx, y_id]
        
        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train,  
                  eval_set=[(X_val, y_val)], 
                  eval_metric=scorer['t1'], 
                  early_stopping_rounds=5)
        feature_imp.append(model.feature_importances_)
        
        tvy.append(model.predict(X_val))
        tty.append(model.predict(X_test))
    vy.append(np.concatenate((tvy[0], tvy[1], tvy[2], tvy[3], tvy[4]), axis=0))
    ty.append(np.mean(np.array(tty), axis=0))

ty = np.array(ty).transpose()
vy = np.array(vy).transpose()
df = pd.DataFrame(ty)
df.to_csv('t1_test.csv', index=False, header=False)
df = pd.DataFrame(vy)
df.to_csv('t1_val.csv', index=False, header=False)
feature_imp = np.array(feature_imp).transpose()
df = pd.DataFrame(feature_imp)
df.to_csv('feature_imp.csv', index=False, header=False)

[0]	validation_0-rmse:0.282786	validation_0-t1e:0.245695
Multiple eval metrics have been passed: 'validation_0-t1e' will be used for early stopping.

Will train until validation_0-t1e hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.274739	validation_0-t1e:0.238883
[2]	validation_0-rmse:0.267177	validation_0-t1e:0.232454
[3]	validation_0-rmse:0.260161	validation_0-t1e:0.226468
[4]	validation_0-rmse:0.253332	validation_0-t1e:0.220569
[5]	validation_0-rmse:0.247062	validation_0-t1e:0.215193
[6]	validation_0-rmse:0.241096	validation_0-t1e:0.21006
[7]	validation_0-rmse:0.235528	validation_0-t1e:0.205191
[8]	validation_0-rmse:0.230447	validation_0-t1e:0.200714
[9]	validation_0-rmse:0.225574	validation_0-t1e:0.196391
[10]	validation_0-rmse:0.220812	validation_0-t1e:0.192147
[11]	validation_0-rmse:0.21651	validation_0-t1e:0.18829
[12]	validation_0-rmse:0.212407	validation_0-t1e:0.184587
[13]	validation_0-rmse:0.208641	validation_0-t1e:0.181132
[14]	validation_0-rmse:0.204993	validation_0-

[139]	validation_0-rmse:0.135585	validation_0-t1e:0.102981
[140]	validation_0-rmse:0.135533	validation_0-t1e:0.102932
[141]	validation_0-rmse:0.135474	validation_0-t1e:0.102859
[142]	validation_0-rmse:0.135426	validation_0-t1e:0.102797
[143]	validation_0-rmse:0.135379	validation_0-t1e:0.102754
[144]	validation_0-rmse:0.135376	validation_0-t1e:0.102748
[145]	validation_0-rmse:0.135353	validation_0-t1e:0.102727
[146]	validation_0-rmse:0.135271	validation_0-t1e:0.102624
[147]	validation_0-rmse:0.135259	validation_0-t1e:0.1026
[148]	validation_0-rmse:0.13518	validation_0-t1e:0.102529
[149]	validation_0-rmse:0.135164	validation_0-t1e:0.10252
[150]	validation_0-rmse:0.135161	validation_0-t1e:0.102504
[151]	validation_0-rmse:0.135115	validation_0-t1e:0.102448
[152]	validation_0-rmse:0.135098	validation_0-t1e:0.102433
[153]	validation_0-rmse:0.135075	validation_0-t1e:0.102416
[154]	validation_0-rmse:0.135032	validation_0-t1e:0.102372
[155]	validation_0-rmse:0.134941	validation_0-t1e:0.102285
[

[279]	validation_0-rmse:0.133148	validation_0-t1e:0.100348
[280]	validation_0-rmse:0.13315	validation_0-t1e:0.100353
[281]	validation_0-rmse:0.133154	validation_0-t1e:0.100348
[282]	validation_0-rmse:0.133146	validation_0-t1e:0.100341
Stopping. Best iteration:
[277]	validation_0-rmse:0.133139	validation_0-t1e:0.10034

[0]	validation_0-rmse:0.27976	validation_0-t1e:0.242752
Multiple eval metrics have been passed: 'validation_0-t1e' will be used for early stopping.

Will train until validation_0-t1e hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.27175	validation_0-t1e:0.235999
[2]	validation_0-rmse:0.264222	validation_0-t1e:0.229613
[3]	validation_0-rmse:0.257221	validation_0-t1e:0.223662
[4]	validation_0-rmse:0.25061	validation_0-t1e:0.218012
[5]	validation_0-rmse:0.244427	validation_0-t1e:0.21272
[6]	validation_0-rmse:0.238492	validation_0-t1e:0.207625
[7]	validation_0-rmse:0.232948	validation_0-t1e:0.202775
[8]	validation_0-rmse:0.227837	validation_0-t1e:0.198309
[9]	validation_

[134]	validation_0-rmse:0.135864	validation_0-t1e:0.103383
[135]	validation_0-rmse:0.135806	validation_0-t1e:0.103299
[136]	validation_0-rmse:0.135747	validation_0-t1e:0.103259
[137]	validation_0-rmse:0.135693	validation_0-t1e:0.10321
[138]	validation_0-rmse:0.135653	validation_0-t1e:0.103172
[139]	validation_0-rmse:0.135609	validation_0-t1e:0.103124
[140]	validation_0-rmse:0.135601	validation_0-t1e:0.103125
[141]	validation_0-rmse:0.135508	validation_0-t1e:0.103011
[142]	validation_0-rmse:0.135512	validation_0-t1e:0.103015
[143]	validation_0-rmse:0.135465	validation_0-t1e:0.102961
[144]	validation_0-rmse:0.135422	validation_0-t1e:0.10292
[145]	validation_0-rmse:0.135393	validation_0-t1e:0.102888
[146]	validation_0-rmse:0.135329	validation_0-t1e:0.102834
[147]	validation_0-rmse:0.135306	validation_0-t1e:0.102797
[148]	validation_0-rmse:0.135234	validation_0-t1e:0.102723
[149]	validation_0-rmse:0.135195	validation_0-t1e:0.102673
[150]	validation_0-rmse:0.135155	validation_0-t1e:0.102616

[274]	validation_0-rmse:0.133398	validation_0-t1e:0.100659
[275]	validation_0-rmse:0.133397	validation_0-t1e:0.100658
[276]	validation_0-rmse:0.13339	validation_0-t1e:0.100653
[277]	validation_0-rmse:0.133356	validation_0-t1e:0.100621
[278]	validation_0-rmse:0.133347	validation_0-t1e:0.100621
[279]	validation_0-rmse:0.133328	validation_0-t1e:0.100601
[280]	validation_0-rmse:0.133318	validation_0-t1e:0.100587
[281]	validation_0-rmse:0.133302	validation_0-t1e:0.100579
[282]	validation_0-rmse:0.133292	validation_0-t1e:0.100571
[283]	validation_0-rmse:0.133287	validation_0-t1e:0.100564
[284]	validation_0-rmse:0.133288	validation_0-t1e:0.100564
[285]	validation_0-rmse:0.133265	validation_0-t1e:0.100528
[286]	validation_0-rmse:0.133271	validation_0-t1e:0.100523
[287]	validation_0-rmse:0.133269	validation_0-t1e:0.100505
[288]	validation_0-rmse:0.133261	validation_0-t1e:0.100492
[289]	validation_0-rmse:0.133261	validation_0-t1e:0.100496
[290]	validation_0-rmse:0.133222	validation_0-t1e:0.10045

[91]	validation_0-rmse:0.139603	validation_0-t1e:0.107411
[92]	validation_0-rmse:0.139487	validation_0-t1e:0.107263
[93]	validation_0-rmse:0.139333	validation_0-t1e:0.107091
[94]	validation_0-rmse:0.139192	validation_0-t1e:0.106936
[95]	validation_0-rmse:0.139026	validation_0-t1e:0.106752
[96]	validation_0-rmse:0.138832	validation_0-t1e:0.106528
[97]	validation_0-rmse:0.138762	validation_0-t1e:0.106448
[98]	validation_0-rmse:0.138643	validation_0-t1e:0.106311
[99]	validation_0-rmse:0.13857	validation_0-t1e:0.106246
[100]	validation_0-rmse:0.138494	validation_0-t1e:0.106169
[101]	validation_0-rmse:0.138371	validation_0-t1e:0.106047
[102]	validation_0-rmse:0.138282	validation_0-t1e:0.105945
[103]	validation_0-rmse:0.138196	validation_0-t1e:0.105861
[104]	validation_0-rmse:0.138083	validation_0-t1e:0.105721
[105]	validation_0-rmse:0.13798	validation_0-t1e:0.105611
[106]	validation_0-rmse:0.137867	validation_0-t1e:0.105459
[107]	validation_0-rmse:0.137824	validation_0-t1e:0.105407
[108]	va

In [ ]:
ty = []
vy = []
feature_imp = []

for y_id in range(3):
    kf = KFold(n_splits=5, shuffle=False)
    tty = []
    tvy = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = Y[train_idx, y_id], Y[val_idx, y_id]
        
        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train,  
                  eval_set=[(X_val, y_val)], 
                  eval_metric=scorer['t2'], 
                  early_stopping_rounds=5)
        feature_imp.append(model.feature_importances_)
        
        tvy.append(model.predict(X_val))
        tty.append(model.predict(X_test))
    vy.append(np.concatenate((tvy[0], tvy[1], tvy[2], tvy[3], tvy[4]), axis=0))
    ty.append(np.mean(np.array(tty), axis=0))

ty = np.array(ty).transpose()
vy = np.array(vy).transpose()
df = pd.DataFrame(ty)
df.to_csv('t2_test.csv', index=False, header=False)
df = pd.DataFrame(vy)
df.to_csv('t2_val.csv', index=False, header=False)
feature_imp = np.array(feature_imp).transpose()
df = pd.DataFrame(feature_imp)
df.to_csv('feature2_imp.csv', index=False, header=False)